# Intorduction

Data includes following signals sampled at 125 Hz: 

(1) Electrocardiogram (ECG): ECG signals represent electrical activity of the heart. ECG signals usually include waves like the P wave (atrial depolarization), QRS complex (ventricular depolarization), and T wave (ventricular repolarization).

(2) Photoplethysmograph (PPG): PPG measures changes in blood volume in the microvascular tissue. PPG waveforms typically include an AC (alternating current) component caused by pulsating pressure and a DC (direct current) component related to baseline blood volume. The key features of PPG signals are the pulse rate and oxygen saturation.

(3) Arterial Blood Pressure (ABP): ABP signals represent the pressure on the walls of arteries caused by blood. The signal includes systolic pressure (maximum pressure during heart contraction), diastolic pressure (minimum pressure during heart relaxation), and mean arterial pressure (average pressure throughout the cardiac cycle).


# Importing libraries and data

In [263]:
import os
import numpy as np 
import pandas as pd
import scipy
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

In [264]:
os.listdir('/kaggle/input/BloodPressureDataset')

['part_4.mat',
 'part_9.mat',
 'part_10.mat',
 'Samples',
 'part_11.mat',
 'part_3.mat',
 'part_1.mat',
 'part_8.mat',
 'part_5.mat',
 'part_6.mat',
 'part_7.mat',
 'part_2.mat',
 'part_12.mat']

In [265]:
os.chdir('/kaggle/input/BloodPressureDataset')

## Visulaization of waveforms

In [266]:
filename = 'part_1.mat'
mat_data = scipy.io.loadmat(filename)['p']

In [267]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True, figsize=(8, 6))
ax1.plot(mat_data[0][0][2][:200])
ax1.set_title('ECG')
ax2.plot(mat_data[0][0][1][:200])
ax2.set_title('PPG')
ax3.plot(mat_data[0][0][0][:200])
ax3.set_title('ABP')

Text(0.5, 1.0, 'ABP')

# Fast Fourier Transform (FFT)

In [275]:
fs = 125
num_samples = len(mat_data[0][0][2])
fft_result_ecg = fft(mat_data[0][0][2])
fft_result_ppg = fft(mat_data[0][0][1])
fft_result_abp = fft(mat_data[0][0][0])
frequencies = fftfreq(num_samples, 1/fs)

fig, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True, figsize=(8, 6))
ax1.plot(frequencies[1:num_samples//2], np.abs(fft_result_ecg)[1:num_samples//2])
ax1.set_title('ECG')
ax2.plot(frequencies[1:num_samples//2], np.abs(fft_result_ppg)[1:num_samples//2])
ax2.set_title('PPG')
ax3.plot(frequencies[1:num_samples//2], np.abs(fft_result_abp)[1:num_samples//2])
ax3.set_title('ABP')
plt.xlim([0, fs/2])
plt.xlabel('Frequency (Hz)')

Text(0.5, 0, 'Frequency (Hz)')

In [284]:
peaks_ecg, _ = find_peaks(np.abs(fft_result_ecg)[1:num_samples//2], distance=num_samples/fs)
peak_index_ecg =  peaks_ecg[np.argsort(np.abs(fft_result_ecg[peaks_ecg]))[::-1][:3]]
peak_frequency_ecg = np.round(frequencies[peak_index_ecg],1)
print('ECG peak freqeuncy')
print(peak_frequency_ecg)

ECG peak freqeuncy
[4.9 8.1 0. ]


In [283]:
peaks_ppg, _ = find_peaks(np.abs(fft_result_ppg)[1:num_samples//2], distance=num_samples/fs)
peak_index_ppg = peaks_ppg[np.argsort(np.abs(fft_result_ppg[peaks_ppg]))[::-1][:3]]
peak_frequency_ppg = np.round(frequencies[peak_index_ppg],1)
print('PPG peak freqeuncy')
print(peak_frequency_ppg)

PPG peak freqeuncy
[1.7 3.2 4.9]


In [285]:
peaks_abp, _ = find_peaks(np.abs(fft_result_abp)[1:num_samples//2], distance=num_samples/fs)
peak_index_abp = peaks_abp[np.argsort(np.abs(fft_result_abp[peaks_abp]))[::-1][:3]]
peak_frequency_abp = np.round(frequencies[peak_index_abp],1)
print('ABP peak freqeuncy')
print(peak_frequency_abp)

ABP peak freqeuncy
[1.7 3.2 0.4]


# Feature Extraction

In [272]:
df = pd.DataFrame()
for part_idx in range(12):
    filename = 'part_'+str(part_idx+1)+'.mat'
    mat_data = scipy.io.loadmat(filename)['p']
    num_signals = mat_data[0].shape[0]
    for ii in range(num_signals):
        ecg_signal = mat_data[0][ii][2]
        ppg_signal = mat_data[0][ii][1]
        abp_signal = mat_data[0][ii][0]
        mean_ecg = np.mean(ecg_signal)
        mean_ppg = np.mean(ppg_signal)
        mean_abp = np.mean(abp_signal)
        mean_ecg = np.mean(ecg_signal)
        mean_ppg = np.mean(ppg_signal)
        mean_abp = np.mean(abp_signal)
        df = df._append(
            {
                'mean_ecg': mean_ecg,
                'mean_ppg': mean_ppg,
                'mean_abp': mean_abp
            }, 
            ignore_index =True
        )